# Verify the Correctness of Exported Model and Compare the Performance

In this tutorial, we are going to show:
- how to verify the correctness of the exported model
- how to compare the performance with the original model

We choose PyTorch to export the ONNX model, and use Caffe2 as the backend.
After that, the outputs and performance of two models are compared.

To run this tutorial, please make sure that `caffe2`, `pytorch`, `onnx`, `onnx-caffe2` are already installed.

First, let's create a PyTorch model and prepare the inputs of the model.

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import io
import numpy as np
import torch
import onnx

from caffe2.proto import caffe2_pb2
from caffe2.python import core
from torch.autograd import Variable
from onnx_caffe2.backend import Caffe2Backend
from onnx_caffe2.helper import c2_native_run_net, name_inputs, save_caffe2_net, load_caffe2_net, \
    benchmark_caffe2_model, benchmark_pytorch_model


class MyModel(torch.nn.Module):
    '''
        This is simple model for demonstration purpose.
        It requires two 2-D tensors as input,
        and returns the multiply of the two inputs.
    '''
    def __init__(self):
        super(MyModel, self).__init__()

    def forward(self, m1, m2):
        return torch.mm(m1, m2)


# Create a pytorch model.
pytorch_model = MyModel()

# Make the inputs in tuple format.
inputs = (Variable(torch.randn(3, 4)), Variable(torch.randn(4, 5)))

Run the PyTorch exporter to generate an ONNX model.

In [3]:
# Export an ONNX model.
f = io.BytesIO()
torch.onnx.export(pytorch_model, inputs, f, verbose=True)
onnx_model = onnx.ModelProto.FromString(f.getvalue())

# Check whether the onnx_model is valid or not.
print("Check the ONNX model.")
onnx.checker.check_model(onnx_model)

graph(%0 : Float(3, 4)
      %1 : Float(4, 5)) {
  %2 : UNKNOWN_TYPE = Constant[value={0}]()
  %3 : Float(3, 5) = Gemm[alpha=1, beta=0, broadcast=1](%0, %1, %2)
  return (%3);
}

Check the ONNX model.


Now, we have an ONNX model, let's turn it into a Caffe2 one.

In [4]:
# Convert the ONNX model to a Caffe2 model.
print("Convert the model to a Caffe2 model.")
init_net, predict_net = Caffe2Backend.onnx_graph_to_caffe2_net(onnx_model.graph)

# Set the device option.
device_opts = core.DeviceOption(caffe2_pb2.CPU, 0)
init_net.device_option.CopyFrom(device_opts)
predict_net.device_option.CopyFrom(device_opts)

Convert the model to a Caffe2 model.


Caffe2 takes a list of numpy array as inputs. So we need to change the format.

In [5]:
# Prepare the inputs for Caffe2.
caffe2_inputs = [var.data.numpy() for var in inputs]

The following code shows how to save and load a Caffe2 model. It is purely for demonstration purpose here.

In [6]:
# Save the converted Caffe2 model in the protobuf files. (Optional)
init_file = "./output/mymodel_init.pb"
predict_file = "./output/mymodel_predict.pb"
save_caffe2_net(init_net, init_file, output_txt=False)
save_caffe2_net(predict_net, predict_file, output_txt=True)

# Load the Caffe2 model.
init_net = load_caffe2_net(init_file)
predict_net = load_caffe2_net(predict_file)

Run PyTorch and Caffe2 models separately, and get the results.

In [7]:
# Compute the results using the PyTorch model.
pytorch_results = pytorch_model(*inputs)

# Compute the results using the Caffe2 model.
_, caffe2_results = c2_native_run_net(init_net, predict_net, name_inputs(onnx_model, caffe2_inputs))

Now we have the results, let's check the correctness of the exported model.
If no assertion fails, our model has achieved expected precision.

In [8]:
# Check the decimal precision of the exported Caffe2.
expected_decimal = 5
for p, c in zip([pytorch_results], caffe2_results):
    if device_opts.device_type == caffe2_pb2.CUDA:
        p.cpu()
    np.testing.assert_almost_equal(p.data.cpu().numpy(), c, decimal=expected_decimal)
print("The exported model achieves {}-decimal precision.".format(expected_decimal))

The exported model achieves 5-decimal precision.


The following code measures the performance of PyTorch and Caffe2 models.
We report:
- Execution time per iteration
- Iterations per second

In [9]:
pytorch_times = benchmark_pytorch_model(pytorch_model, inputs)
caffe2_times = benchmark_caffe2_model(init_net, predict_net)

print("PyTorch model's execution time is {} / iteration.".format(pytorch_times[0]))
print("Caffe2 model's execution time is {} / iteration.".format(caffe2_times[0]))

PyTorch model's execution time is 0.00998973846436 / iteration.
Caffe2 model's execution time is 0.00594469998032 / iteration.
